In [1]:
import tensorflow as tf
import pickle
import sys
import matplotlib.pyplot as plt1 
import numpy as np
from IPython.display import Audio, display
import time
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate, Conv1D,Conv2D, Flatten, Reshape, Embedding, GRU, SpatialDropout1D, LSTM, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from itertools import permutations
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy.stats import trim_mean

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
full = pd.read_csv("../data/drug_class_identification/all3.csv")
full = full.dropna()
full['atc'] = full['atc'].apply(lambda x : x[0])
full.head()

,smiles,name,id,inchi_key,atc
75,NC12CC3CC(CC(C3)C1)C2,amantadine,BRD-K70330367,DKNWSYNQZKUICI-UHFFFAOYSA-N,N
291,CS(=O)(=O)OCCCCOS(C)(=O)=O,busulfan,BRD-K23204545,COVZYZSDYWQREU-UHFFFAOYSA-N,L
322,CCCC(C)(COC(N)=O)COC(=O)NC(C)C,carisoprodol,BRD-A99939097,OFZCIYFFPZCNJE-UHFFFAOYSA-N,M
491,CCN(CC)C(=O)N1CCN(C)CC1,diethylcarbamazine,BRD-K45542189,RCKMWOKWVGPNJF-UHFFFAOYSA-N,P
510,CCN(CC)C(=S)SSC(=S)N(CC)CC,disulfiram,BRD-K32744045,AUZONCFQVSMFAP-UHFFFAOYSA-N,P


In [3]:
# Define our own plot function
def scatter(x, y, subtitle=None):
    le = LabelEncoder()
    labels = le.fit_transform(y)

    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 14))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(14):
        # Position of each label.
        xtext, ytext = trim_mean(x[labels == i, :], axis=0, proportiontocut=0.2)
        letter = le.inverse_transform([i])[0]
        txt = ax.text(xtext, ytext, str(letter), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [4]:
snn = pd.read_csv('EMB_snn_4_25_16_full-75')
snn['id'] = snn['pert_id']
result_snn = pd.merge(snn, full, how="inner").drop(['id','smiles','name','inchi_key','pert_id','Unnamed: 0'],axis=1).dropna()

In [5]:
y_snn = result_snn['atc']
X_snn = result_snn.drop(['atc'],axis=1)
X_snn = np.asarray(X_snn)
result_snn.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,atc
0,0.350203,0.272261,0.322720,-0.415728,-0.118934,-0.274651,-0.195480,0.411698,0.002516,-0.112954,0.226189,-0.052496,0.047790,0.034436,-0.138315,0.373745,D
1,0.350203,0.272261,0.322720,-0.415728,-0.118934,-0.274651,-0.195480,0.411698,0.002516,-0.112954,0.226189,-0.052496,0.047790,0.034436,-0.138315,0.373745,G
2,0.008870,0.203011,0.045596,-0.303917,-0.486009,0.060476,0.099339,0.431585,-0.201203,-0.458416,-0.011151,-0.193102,0.009726,-0.025093,-0.182268,0.326008,D
3,0.008870,0.203011,0.045596,-0.303917,-0.486009,0.060476,0.099339,0.431585,-0.201203,-0.458416,-0.011151,-0.193102,0.009726,-0.025093,-0.182268,0.326008,G
4,0.054629,0.270735,0.245764,-0.271058,-0.155077,0.056891,-0.155674,0.369895,0.163061,-0.047226,0.473894,-0.323181,0.010434,-0.224793,-0.183408,0.399229,D


In [6]:
X_snn.shape

(49373, 16)

In [7]:
x_train_snn, x_test_snn, y_train_snn, y_test_snn = train_test_split(X_snn,y_snn)

x_train_flat_snn = x_train_snn.reshape(-1,16)
x_test_flat_snn = x_test_snn.reshape(-1,16)

tsne = TSNE()
train_tsne_embeds_snn = tsne.fit_transform(x_train_flat_snn[:500])
scatter(train_tsne_embeds_snn, y_train_snn[:500], "Samples from Training Data LINCS (DenseNet)")

eval_tsne_embeds_snn = tsne.fit_transform(x_test_flat_snn[:500])
scatter(eval_tsne_embeds_snn, y_test_snn[:500], "Samples from Validation Data LINCS (DenseNet)")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


In [8]:
def generateTriplet(x,y,testsize=0.2,ap_pairs=10,an_pairs=10):
    data_xy = tuple([x,y])

    trainsize = 1-testsize

    triplet_train_pairs = []
    triplet_test_pairs = []
    for data_class in sorted(set(data_xy[1])):

        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        A_P_pairs = random.sample(list(permutations(same_class_idx,2)),k=ap_pairs) #Generating Anchor-Positive pairs
        Neg_idx = random.sample(list(diff_class_idx),k=an_pairs)
        

        #train
        A_P_len = len(A_P_pairs)
        Neg_len = len(Neg_idx)
        for ap in A_P_pairs[:int(A_P_len*trainsize)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_train_pairs.append([Anchor,Positive,Negative])               
        #test
        for ap in A_P_pairs[int(A_P_len*trainsize):]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_test_pairs.append([Anchor,Positive,Negative])    
                
    return np.array(triplet_train_pairs), np.array(triplet_test_pairs)

In [9]:
X_train_snn, X_test_snn = generateTriplet(X_snn[:50],y_snn[:50])

In [93]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    print('y_pred.shape = ',y_pred)
    
    total_length = y_pred.shape.as_list()[-1]
    
    anchor = y_pred[:,:,0:14]
    positive = y_pred[:,:,14:28]
    negative = y_pred[:,:,28:]

    print(anchor,'\n', positive,'\n', negative, '\n',y_pred)
    print(anchor.shape, positive.shape, negative.shape)
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

def baseNetwork():    
        model = Sequential()
        model.add(Dense(1000, activation='softmax'))
        model.add(Dropout(0.9))
        model.add(Dense(1000, activation='softmax'))
        model.add(Dropout(0.9))
        model.add(Dense(14, activation='softmax'))
        return model

In [94]:
anchor_input = Input((16,1, ), name='anchor_input')
positive_input = Input((16,1, ), name='positive_input')
negative_input = Input((16,1, ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = baseNetwork()
encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)

merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative],axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer='adam')
model.summary()

y_pred.shape =  Tensor("merged_layer_18/concat:0", shape=(?, 16, 42), dtype=float32)
Tensor("loss_17/merged_layer_loss/strided_slice:0", shape=(?, 16, 14), dtype=float32) 
 Tensor("loss_17/merged_layer_loss/strided_slice_1:0", shape=(?, 16, 14), dtype=float32) 
 Tensor("loss_17/merged_layer_loss/strided_slice_2:0", shape=(?, 16, 14), dtype=float32) 
 Tensor("merged_layer_18/concat:0", shape=(?, 16, 42), dtype=float32)
(?, 16, 14) (?, 16, 14) (?, 16, 14)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 16, 1)        0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 16, 1)        0                                            
_______________________________________________

In [95]:
Anchor_snn = X_train_snn[:,0,:].reshape(-1,16,1)
Positive_snn = X_train_snn[:,1,:].reshape(-1,16,1)
Negative_snn = X_train_snn[:,2,:].reshape(-1,16,1)
Anchor_test_snn = X_test_snn[:,0,:].reshape(-1,16,1)
Positive_test_snn = X_test_snn[:,1,:].reshape(-1,16,1)
Negative_test_snn = X_test_snn[:,2,:].reshape(-1,16,1)

Y_dummy = np.empty((Anchor_snn.shape[0],300))
Y_dummy2 = np.empty((Anchor_test_snn.shape[0],1))

model.fit([Anchor_snn,Positive_snn,Negative_snn],y=Y_dummy,validation_data=([Anchor_test_snn,Positive_test_snn,Negative_test_snn],Y_dummy2), epochs=100)

Train on 160 samples, validate on 40 samples
Epoch 1/100
160/160 [==============================] - 5s 30ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 2/100
160/160 [==============================] - 2s 12ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 3/100
160/160 [==============================] - 2s 10ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 4/100
160/160 [==============================] - 2s 10ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 5/100
160/160 [==============================] - 2s 12ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 6/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 7/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 8/100
160/160 [==============================] - 2s 13ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 9/100
160/160 [==============================] - 3s 19ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 10/100
160/160 [===============

160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 81/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 82/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 83/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 84/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 85/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 86/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 87/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 88/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - val_loss: 0.4000
Epoch 89/100
160/160 [==============================] - 2s 11ms/step - loss: 0.4000 - 

In [96]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

In [97]:
tsne = TSNE()
X_train_trm = trained_model.predict(x_train_snn[:100].reshape(-1,16,1))
X_test_trm = trained_model.predict(x_test_snn[:100].reshape(-1,16,1))

In [98]:
X_test_trm

array([[[0.071298  , 0.07152329, 0.07134153, ..., 0.07130028,
         0.07155156, 0.07131519],
        [0.071298  , 0.07152329, 0.07134153, ..., 0.07130028,
         0.07155156, 0.07131519],
        [0.071298  , 0.07152329, 0.07134152, ..., 0.07130028,
         0.07155156, 0.07131518],
        ...,
        [0.071298  , 0.07152329, 0.07134153, ..., 0.07130028,
         0.07155156, 0.07131519],
        [0.071298  , 0.07152329, 0.07134153, ..., 0.07130028,
         0.07155156, 0.07131518],
        [0.071298  , 0.07152329, 0.07134152, ..., 0.07130028,
         0.07155155, 0.07131518]],

       [[0.071298  , 0.07152329, 0.07134152, ..., 0.07130028,
         0.07155156, 0.07131518],
        [0.071298  , 0.07152329, 0.07134152, ..., 0.07130028,
         0.07155156, 0.07131518],
        [0.071298  , 0.07152329, 0.07134152, ..., 0.07130028,
         0.07155155, 0.07131518],
        ...,
        [0.071298  , 0.07152329, 0.07134153, ..., 0.07130028,
         0.07155156, 0.07131518],
        [0.0